## 5.1 Python's Sequence Types 

- In this chapter, we explore Python's various "sequence" classes, namely the built-in list, tuple, and str classes. There is significant commonality between these classes, most notably: each supports indexing to access an individual element of sequence, using syntax such as seq[k], and each uses a low-level comcept knwon as an array to represent the sequence. However, there are significant differences in the abstractions that these classes represent, and in the way that instances of these classes are represented internally by Python. Because these classes are used so widely in PYthon programs, and because they will become building blocks upon which we will develop more complex data structures, it is imperative that we establish a clear understanding of both the public behavior and inner workings of these classes.

** Public Behaviors **

- A proper understanding of  the outward semantics for a class is a necessity for a good programmer. While the basic usage of lists, strings, and tuples may seem straightforward, there are several important subtleties regarding the behaviors associatedd with these classes (such as what it means to make a copy of a sequence, or to take a slice of a sequence). Having a misunderstanding of a behavior can easily lead to inadvertent bugs in a program. Therefore, we establish an accurate mental model for each of these classes. These images will help when exploring more advanced usage, such as representing a multidimensional data set as a list of lists.

** Implementation Details **

- A focus on the internal implementations of these classes seems to go against our stated principles of object-oriented programming. In Section 2.1.2, we emphasized the principle of encapsulation, noting that the user of a class need not know about the internal details of the implementations. While it is true that one only needs to understand the syntax and semantics of class's public interface in order to be able to write legal and corrct code that uses instances of the class, the efficiency of a program depends greatly on the efficiency of the components which it relies.

** Asymptotic and Experimental Analyses **

- In describing the efficiency of various operations for Python's sequence classes, we will rely on the formal asymptotic analysis notations established on Chapter 3. We will also perform experimental analyses of the primary operations to provide empirical evidence that is consistent with the more theoretical asymptotic analyses.

## 5.2 Low-Level Arrays

- To accurately describe the way in which Python represents the sequence types, we must first discuss aspects of the low-level computer architecture. The primary memory of a computer is composed of bits of information, and those bits are typically grouped into larger units that depend upon the precise system architecture. Such a typical unit is a byte which is equivalent to 8 bits.

- A computer system will have a huge number of bytes of memory, and to keep track of what information is stored in what byte, the computer uses an abstraction known as a memory address. In effect, each byte of memory is associated with a unique number that serves as its address(more formally, the binary representation of the number serves as the address). In this way, the computer system can refer to the data in "byte #2150" versus the data in "byte #2157", for example. Memory addresses are typically coordinated with the physical layout of the memory system, and so we often portray the numbers in sequential fashion. Figure 5.1 provides such a diagram, with the designated memory address for each byte.

- Despite the sequential nature of the numbering system, computer hardware is designed, in theory, so that any byte of the main memory can be efficiently accessed based upon its memory address. Im this sense, we say that computer's main memory performs as random access memory(RAM). That is, it is just as easy to retrieve byte #8675309 as it is to retrieve byte #309.(In practice, there are complicating factors including the use of caches and external memory; we address some of those issues in Chapter 15.) Using the notation for asymptotic analysis, we say that any individual byte of memory can be stored or rerieved in O(1) time.

- In general, a programming language keeps track of the association between an identifier and the memory address in which the associated value is stored. For example, identifier x might be associated with one value stored in memory, while y is associated with another value stoed in memory. A common programming task is to keep track of a sequence of related objects. For example, we may want a video game to keep track of the top ten scores for that game. Rather than use ten different variables for this task, we would prefer to use a single name for the group and use index numbers to refer to the high scores in that group.

- A group of related variables can be stored one after another in contiguous portion of the computer's memory. We will denote such a representation as an array. As a tangible example, a text string is stored as an oredered sequence of individual characters. In Python, each character is represented using the Uniode character set, and on most computing systems, Python internally represents each Unicode character with 16 bits(i.e., 2 bytes). Therefore, a six-character string, such as "SAMPLE", would be stored in 12 consecutive bytes of memory, as diagrammed in Figure 5.2.

- We describe this as an array of six characters, even though it requires 12 bytes of memory. We will refer to each location within an array as a cell, and will use an integer index to describe its location within the array, with cells numbered starting with 0, 1, 2, and so on. For example, in Figure 5.2, the cell of the array with index 4 has contents L and is stored in bytes 2154 and 2155 of memory.

- Each cell of an array must use the same number of bytes. This requirement is what allows an arbitrary cell of the array to be accessed in constant time based on its index. In particular, if one knows the memory address at which an array starts (e.g., 2146 in Figure 5.2), the number of bytes per element (e.g. 2 for  a Unicode character), and a desired index within the array, the appropriate memory address can be computed using the calculation, start + cellsize \* index.  By this formula, the cell at index 0 begins precisely at the start of the array, the cell at index 1 begins precisely cellsize bytes beyond the start of the array, and so on. As on example, cell 4 of Figure 5.2 begins at memory location 2146 + 2 \* 4 = 2146 + 8 = 2154.

- Of course, the arithmetic for calculating memory addresses within an array can be handled automatically. Therefore, a programmer can envision a more typical high-level abstraction of an array of characters as diagrammed in Figure 5.3.

### 5.2.1 Referential Arrays

- AS another motivating example, assume that we want a medical information system to keep track of the patients currently assigned to beds in a certain hospital. If we assume that the hospital has 200 beds, and converniently that those beds are numbered from 0 to 199, we mgiht consider using an array-based structure to maintain the names of the patients currently assigned to those beds. For example, in Python we might use a list of names, such as:

['Rene', 'Joseph', 'Janet', 'Jonas', 'Helen', 'Virginia', ///]

- To represent such a list with an array, Python must adhere to the requirement that each cell of the array use the same number of bytes. Yet the elements are strings, and strings naturally have different lengths. Python could attempt to reserve enough space for each cell to hold the maximum length string (not just of currently stored strings, but of any string we might ever want to store), but that would be wasteful.

- Instead, Python represents a list or tuple instance using an internal storage mechanism of an array of object references. At the lowest level, what is stored is a consecutive sequence of memory addresses at which the elements of the sequence reside. A high-level diagram of such a list is shown in Figure 5.4.

- Although the relative size of the individual elements may vary, the number of its used to store the memoty address of each element is fixed (e.g., 64-bits per address). In this way, Python can support contant-time access to a list or tuple element based on its index.

- In Figure 5.4, we characterize a list of strings that are the names of the patients in a hospital. Is is more likely that a medical information system would manage more comprehensive information on each patient, perhaps represented as an instance of a Patient class. From the perspective of the list implementation, the same principle applies: The list will simply kepp a sequence of references to those objects. Note as well that a refenrence to the None object can be used as an element of the list to represent an empty bed in the hospital.

- The fact that list and tuples are referential structures is significant to the semantics of these classes. A single list instance may include multiple references to the same object as elements of the list, and it is possible for a single object to be an element of two or more lists, as those lists simply store references back to that object. As an example, when you compute a slice of a list, the result is a new list instance, but that new list has references to the same elements that are in the original list, as portrayed in Figure 5.5.

- When the elements of the list are immutable objects, as with the integer instances in Figure 5.5, the fact that the two lists share elements is not that significant, as neither of the lists can cause a change to the shared object. If, for example, the commnad temp[2] = 15 were executed from this configuration, that does not change the existing integer object; it changes the reference in cell 2 of the temp list to reference a different object. The resulting configuration is shown in Figure 5.6.

- The same semantics is demonstrated when making a new list as a copy of an existing one, with a syntax such as backup = list(primes). This produces a new list that is a shallow copy, in that it references the same elements as in the first list. With immutable elements, this point is moot. If the contents of the list were of a mutable type, a deep copy, meaning a new list with new elements, can be produced by using the deppcopy function from the copy module.

- AS a more striking example, it is a common practice in Python to initialize an array of integers using a syntax such as counters = [0] \* 8. This syntax produces a list of length eight, with all eight elements being the value zero. Technically, all eight cells of the list reference the same object, as portrayed in Figure 5.7.

- At first glance, the extreme level of aliasing in this configuration may seem alarming. However, we rely on the fact that the referenced integer is immutable. Even a commnad such as counters[2] += 1 does not technically change the value of the existing integer insatance. This computes a new integer, with value 0 + , and sets cell 2 to reference the newly computed value. The resulting configuration is shown in Figure 5.8.

![main](no_new_elements.png "main")

- AS a final manifestation of the referential nature of lists, we note that the extend commnad is used to add all elements from one list to the end of another list. The extended list does not receive copies of those elements, is receives references to those elements. Figure 5.8 portrays the effect of a call to extend.

p 212